# 带式输送机4-D

## 1. 传动装置的总体设计

输送带的牵引力: $F_w = 3.2 \text{ kN}$

输送带的速度: $v_w = 0.95 \text{ m/s}$

输送带滚筒的直径: $D = 440 \text{ mm}$

In [ ]:
import numpy as np

# 工作轴
F_w = 3.2 # kN
v_w = 0.95 # m/s
D = 440 # mm


计算输出轴的参数

输出轴工作力矩: $T = F_w \cdot D / 2$

输出轴工作转速: $\omega = \frac{v_w}{\frac{D}{2}\times 10^{-3}}$

输出轴工作功率: $P_w = T \cdot \omega$


In [8]:
# 工作力矩
T = F_w * D / 2 # N * m
# 工作转速
omega = v_w / (D / 2 * 0.001) # rad/s
# 工作功率
P_w = T * omega # W
print('工作力矩:{:.3f} N*m , 工作转速:{:.3f} rad/s, 工作功率:{:.3f} W'.format(T, omega, P_w))

工作力矩:704.000 N*m , 工作转速:4.318 rad/s, 工作功率:3040.000 W


工作转速: $n_w = \frac{60 \omega}{2 \pi}$

In [10]:
n_w = 60 * omega / (2 * np.pi)
print('工作转速：{:.3f} rpm'.format(n_w))

工作转速：41.236 rpm


接下来计算工作机所需功率, 我们要考虑图中的效率

<img src="./fig/f1.jpg" width="25%" height="25%" align = "center">

从电动机到工作机的总效率为$\eta = \prod \eta_i$

In [11]:
# 13页表格
cls2eta = {
    '带': 0.96,
    '联轴器': 0.99,
    '链': 0.96,
    '轴承': 0.98,
    '齿轮': 0.98,
}

# 简图中统计
cls_count = {
    '带': 1,
    '联轴器': 2,
    '链': 1,
    '轴承': 3,
    '齿轮': 2,
}

eta = 1
for cls, count in cls_count.items():
    eta *= cls2eta[cls] ** count
    
print('总效率为: {:.3f}'.format(eta))

总效率为: 0.816


电机所需功率: $P_m = P_w / \eta$

In [13]:
P_m = T * omega / eta
print('电机所需功率为：{:.3f} W'.format(P_m))

电机所需功率为：3723.320 W


于是根据196页,我们选取电机型号: YE4-112M-4

额定功率为$4\text{kW}$, 满载转速为$1455 \text{r/min}$, 同步转速为: $n_{m} = 1500 \text{rpm}$, 轴直径为$\Phi 28j6$

In [23]:
n_m = 1455
i = n_m / n_w
print('总传动比为:{:.3f}'.format(i))

总传动比为:35.285


因为我们需要设计的是一个 _二级圆柱齿轮减速器_

根据15页的经验公式, 我们得到了两个传动比

$i_1 = \sqrt{1.3i}$

$i_2 = i / i_1$

In [24]:
i_1 = np.sqrt(1.3 * i)
i_2 = i / i_1
print('i1 = {:.3f}, i2 = {:.3f}'.format(i_1, i_2))

i1 = 6.773, i2 = 5.210


我们记从电动机到工作机为正向, 减速器的三个轴分别为1,2,3轴,则它们的转速为

轴1的转速:$n_1 = n_m$

轴2的转速:$n_2 = n_1 / i_1$

轴3的转速:$n_3 = n_2 / i_2$

In [27]:
n1 = n_m
n2 = n1 / i_1
n3 = n2 / i_2
print('轴1的转速为:{:.3f}, 轴2的转速为:{:.3f}, 轴3的转速为:{:.3f}'.format(n1, n2, n3))

轴1的转速为:1455.000, 轴2的转速为:214.831, 轴3的转速为:41.236


各轴的输入功率为

$\eta_1 = \eta_{联轴器} \cdot \eta_{轴承}$

$P_1 = P_m \cdot \eta_1$

$\eta_2 = \eta_{齿轮} \cdot \eta_{轴承}$

$P_2 = P_1 \cdot \eta_2$

$\eta_3 = \eta_{齿轮} \cdot \eta_{轴承}$

$P_3 = P_2 \cdot \eta_3$

In [28]:
eta_1 = cls2eta['联轴器'] * cls2eta['轴承']
eta_2 = cls2eta['齿轮'] * cls2eta['轴承']
eta_3 = cls2eta['齿轮'] * cls2eta['轴承']
P_1 = P_w * eta_1
P_2 = P_1 * eta_2
P_3 = P_2 * eta_3
print('P1 = {:.3f} W, P2 = {:.3f} W, P3 = {:.3f} W'.format(P_1, P_2, P_3))

P1 = 2949.408 W, P2 = 2832.611 W, P3 = 2720.440 W


各轴的输入转矩

$T_1 = 9.55 P_1 / n_1$

$T_2 = 9.55 P_2 / n_2$

$T_3 = 9.55 P_3 / n_3$

In [60]:
T_1 = 9.55 * P_1 / n1
T_2 = 9.55 * P_2 / n2
T_3 = 9.55 * P_3 / n3
print('T1 = {:.3f} N*m, T2 = {:.3f} N*m, T3 = {:.3f} N*m'.format(T_1, T_2, T_3))

T1 = 19.359 N*m, T2 = 125.920 N*m, T3 = 630.043 N*m


## 2. 传动零件设计和联轴器的选择

### 2.1. 初算轴径

机械设计381页公式

扭转切应力: $\tau_{T} = \frac{9550 P/n}{0.2 d^3} \le [\tau_{T}]$

于是:

$d \ge A_0 (\frac{0.001P}{n})^{1/3}$

选取轴材料为<font color='red'>45钢</font>

查382页表, 由于只受扭矩, $A_0$取较小值, 所以取 $A_0 = 103$

于是, 我们得到了各轴的最小轴径估计

In [61]:
# 查表
A0 = 103
d1 = A0 * (0.001 * P_1/n1)**(1/3)
d2 = A0 * (0.001 * P_2/n2)**(1/3)
d3 = A0 * (0.001 * P_3/n3)**(1/3)

print('d1 = {:.3f} mm, d2 = {:.3f} mm, d3 = {:.3f} mm'.format(d1, d2, d3))

d1 = 13.036 mm, d2 = 24.333 mm, d3 = 41.619 mm


各轴都安装有一个大齿轮,键槽数都是1, 都增大$7\%$

In [ ]:
d1 *= 1.07
d2 *= 1.07
d3 *= 1.07
print(d1,d2,d3)

## 2.2 齿轮设计

### 2.2.1 初步设计

精度等级: 由机械设计216页, 带式输送机为一般工作机器, 选用7级精度.

材料选择: 由机械设计203页, 选择小齿轮材料为40Cr(调质),齿面硬度 $280$ HBW;大齿轮材料为$45$钢(调质),齿面硬度 $240$ HBW.

压力角$\alpha$的选择: 机械设计216页, 压力角选择$\alpha = 20^\circ$

齿宽系数 $\phi_d$ 的选择: 根据机械设计216页表10-8, 我们的装置情况为"两支承相对于小齿轮作不对称布置", 我们选取$\phi_d = 0.8$.

In [62]:
alpha = np.deg2rad(20)
print('alpha = {:.3f} rad'.format(alpha))
phi_d = 0.8
print('phi_d = {:.3f}'.format(phi_d))

alpha = 0.349 rad
phi_d = 0.800


### 2.2.2 齿数$z$的选择

机械设计216页, 选 $z_1=17$, $z_2' = 20 $

则:

$z_2 = \lceil z_1 i_1 \rceil$

$z_3 = \lceil z_2' i_2 \rceil$

In [63]:
z1_small = 17
z2_small = 20
z2 = int(np.ceil(z1_small * i_1))
z3 = int(np.ceil(z2_small * i_2))
print('z2 = {}, z3 = {}'.format(z2, z3))

z2 = 116, z3 = 105


### 2.2.3 试算小齿轮分度圆直径

我们按照齿面接触疲劳强度设计齿轮的分度圆直径.

根据机械设计214页公式

$d \ge \sqrt[3]{\frac{2K_{Ht} T}{\phi_{d}} \frac{u + 1}{u} (\frac{Z_H Z_E Z_{\varepsilon}}{[\sigma_H]})^2 }$

其中$T$为小齿轮传递的转矩, 即大齿轮轴的输入转矩

于是我们只需要确定$K_{Ht},u,Z_H,Z_E,Z_{\varepsilon},[\sigma_H]$

&ensp; 接触疲劳强度计算的载荷系数: $K_{Ht}$ 试选 $K_{Ht}=1.3$

&ensp; 齿数比: $u = \frac{z_2}{z_1}$

&ensp; 区域系数: $Z_H = \frac{2}{\cos\alpha \sin\alpha}$

&ensp; 弹性影响系数: $Z_E=188 \text{MPa}^{1/2}$

&ensp; 重合度系数 $Z_{\varepsilon}$:

&ensp; &ensp; 首先计算齿顶压力角: $\alpha_{a} = \arccos(\frac{z\cos\alpha}{z+2h_a^{*}})$

&ensp; &ensp; 然后计算重合度: $\varepsilon_{\alpha} = \frac{z_1(\tan\alpha_{a1}-\tan\alpha)+z_2(\tan\alpha_{a2}-\tan\alpha)}{2\pi}$

&ensp; &ensp; 最后得到重合度系数: $Z_{\varepsilon} = \sqrt{\frac{4 - \varepsilon_{\alpha}}{3}}$

&ensp; 接触疲劳许用应力 $[\sigma_H]$:

&ensp; &ensp; 首先计算应力循环次数$N$: $N = 60njL_h$

&ensp; &ensp; &ensp; 齿轮每转一圈,同一齿面的啮合次数: $j = 1$

&ensp; &ensp; &ensp; 齿轮的工作寿命(hour): $L_h = 8\text{小时/班}\times 2\text{班/天} \times 300 \text{天/年} \times 10 \text{年} = 48000 \text{小时}$

&ensp; &ensp; &ensp; 循环次数: $N = 2880000n$, 三个轴的循环次数分别为$4\times 10^9, 6\times 10^8, 1\times 10^8$

&ensp; &ensp; 疲劳寿命系数$K_{HN}$: 查机械设计218页10-19表, 三个轴分别为$K_{HN1}=0.89, K_{HN2}=0.93, K_{HN3}=0.97$

&ensp; &ensp; 接触疲劳极限: 由机械设计221页图c得, 小齿轮$\sigma_{Hlim1}=600 \text{MPa}$, 大齿轮$\sigma_{Hlim2}=550 \text{MPa}$

&ensp; &ensp; 取失效概率为$1\%$, 安全系数$S=1$, 由机械设计217页公式(10-14)得: $[\sigma]=\frac{K_N \sigma_{lim}}{S}$

&ensp; &ensp; 计算得到, 轴1与轴2接触副的许用应力为$\sigma_{H12} = 551.5 \text{MPa}$, 轴2与轴3接触副的许用应力为$\sigma_{H23} = 533.5 \text{MPa}$

In [64]:
# input: n1,z1,n2,z2
def cal_N(n):
    return 2880000 * n

print('轴1循环次数: {:e}, 轴2循环次数: {:e}, 轴3循环次数: {:e}'.format(cal_N(n1),cal_N(n2),cal_N(n3)))

轴1循环次数: 4.190400e+09, 轴2循环次数: 6.187119e+08, 轴3循环次数: 1.187585e+08


In [65]:
N1 = cal_N(n1)
N2 = cal_N(n2)
N3 = cal_N(n3)

# 查表
K_HN1 = 0.89
K_HN2 = 0.93
K_HN3 = 0.97
S = 1
sigma_Hlim_small = 600
sigma_Hlim_big = 550

def cal_sigma_H(k_HN1, k_HN2):
    sigma_H1 = k_HN1 * sigma_Hlim_small / S
    sigma_H2 = k_HN2 * sigma_Hlim_big / S
    sigma_H = min(sigma_H1, sigma_H2)
    return sigma_H

sigma_H_small_1 = cal_sigma_H(K_HN1, K_HN2)
sigma_H_small_2 = cal_sigma_H(K_HN2, K_HN3)
print('轴1上的小齿轮许用应力={:.3f} MPa, 轴2上的小齿轮许用应力={:.3f} MPa'.format(sigma_H_small_1, sigma_H_small_2))

轴1上的小齿轮许用应力=511.500 MPa, 轴2上的小齿轮许用应力=533.500 MPa


In [66]:
ha_star = 1
pi = np.pi
# global alpha
# input: z1, z2, alpha
def cal_epsilon_alpha(z1, z2):
    '''计算重合度'''
    # 齿顶压力角
    alpha_a1 = np.arccos(z1 * np.cos(alpha)/ (z1 + 2*ha_star))
    alpha_a2 = np.arccos(z2 * np.cos(alpha)/ (z2 + 2*ha_star))
    # 重合度
    epsilon_alpha = (z1*(np.tan(alpha_a1) - np.tan(alpha)) + z2*(np.tan(alpha_a2) - np.tan(alpha)))/(2*pi)
    return epsilon_alpha

epsilon_alpha = cal_epsilon_alpha(z1_small, z2)
print('重合度 = {:.3f}'.format(epsilon_alpha))

# 重合度系数
Z_epsilon = np.sqrt((4-epsilon_alpha)/3)
print('重合度系数Z_epsilon = {:.3f}'.format(Z_epsilon))

def cal_Z_epsilon(z1,z2):
    epsilon_alpha = cal_epsilon_alpha(z1, z2)
    Z_epsilon = np.sqrt((4-epsilon_alpha)/3)
    return Z_epsilon

重合度 = 1.691
重合度系数Z_epsilon = 0.877


In [77]:
K_Ht = 1.3

def design_d(z1,z2,sigma_H, T1):
    u = z2 / z1
    Z_H = 2 / (np.cos(alpha) * np.sin(alpha))
    Z_E = 188
    Z_epsilon = cal_Z_epsilon(z1,z2)
    d = (2*K_Ht*1000*T1/(phi_d) * (u+1)/u * (Z_H*Z_E*Z_epsilon/sigma_H)**2)**(1/3)
    return d

d_1_t = design_d(z1_small,z2,sigma_H_small_1, T_1)
d_2_t = design_d(z2_small,z3,sigma_H_small_2, T_2)
print('d1t = {:.3f} mm, d2t= {:.3f} mm'.format(d_1_t,d_2_t))


d1t = 66.221 mm, d2t= 121.428 mm


### 2.2.4 调整小齿轮分度圆直径

圆周速度: $v = \frac{\pi d_t n}{60\times 1000}$

齿宽: $b=\phi_d d_t$

实际载荷系数 $K_H$ : 

由机械设计205页表10-2查得齿间载荷分配系数 $K_A$

由机械设计206页图10-8,根据$v$和7级精度选取动载荷系数 $K_v$

齿间载荷分配系数 $K_{H\alpha}$

&ensp; $F_{t} = \frac{2T}{d_t}$

&ensp; $\frac{K_AF_t}{b}= ? 100 \text{N/mm}$

查机械设计207页表10-3得齿间载荷分配系数 $K_{H\alpha}=$

由表机械设计208页表10-4对$b$用插值法查得7级精度、小齿轮相对支撑非对称布局时的齿向载荷分布系数 $K_{H\beta}=$

由此, 得到实际载荷系数

$$K_H = K_A K_v K_{H\alpha} K_{H\beta} =$$

由机械设计215页公式10-12可得按实际载荷系数算得的分度圆直径:

$$
d_{H} = d_t \sqrt[3]{\frac{K_H}{K_{Ht}}}
$$

In [87]:
K_A = 1

def print_v_b(dt, n, T):
    v = pi * dt * n / (60 * 1000)
    b = phi_d * dt
    Ft = 2 * T / (0.001 * dt)
    less_than_100 = K_A * Ft / b < 100
    print('v={:.2f} m/s, b={:.2f} mm, less_than_100={}'.format(v,b,less_than_100))
    return b

b1 =print_v_b(d_1_t, n1, T_1)
b2 = print_v_b(d_2_t, n2, T_2)

v=5.04 m/s, b=52.98 mm, less_than_100=True
v=1.37 m/s, b=97.14 mm, less_than_100=True


轴1:$K_v = 1.14$, $K_{H\alpha}=1.2$

轴2:$K_v = 1.05$, $K_{H\alpha}=1.2$

In [91]:

# 查表
K_v1 = 1.14
K_v2 = 1.05
K_Ha1 = 1.2
K_Ha2 = 1.2

def interpolation(x1,y1,x2,y2,x):
    k = (y2-y1)/(x2-x1)
    dx = x - x1
    y = k*dx + y1
    return y

# 查表
K_Hb1 = interpolation(40,1.201,80,1.210,b1)
K_Hb2 = interpolation(80,1.210,120,1.219,b2)
print('KHb1={:.3f}, KHb2={:.3f}'.format(K_Hb1,K_Hb2))


KHb1=1.204, KHb2=1.214


In [92]:
K_H_1 = K_A * K_v1 * K_Ha1 * K_Hb1
K_H_2 = K_A * K_v2 * K_Ha2 * K_Hb2
print('KH1={:.3f}, KH2={:.3f}'.format(K_H_1, K_H_2))

KH1=1.647, KH2=1.529


In [130]:
K_H_t1 = K_Ht
K_H_t2 = K_Ht

d_1_H = d_1_t * (K_H_1 / K_H_t1)**(1/3)
d_2_H = d_2_t * (K_H_2 / K_H_t2)**(1/3)
print('d1H={:.3f} mm, d2H={:.3f} mm'.format(d_1_H, d_2_H))

d1H=71.654 mm, d2H=128.188 mm


相应的齿轮模数 $m_H = \frac{d_{H}}{z}$

In [131]:
m_H = d_1_H / z1_small
print('m_H = {:.3f}'.format(m_H))

m_H = 4.215


### 2.2.5 按齿根疲劳强度设计

模数

$$
m_t \ge \sqrt[3]{\frac{2K_{Ft}T_1Y_{\varepsilon}}{\phi_d z_1^2} \frac{Y_{Fa}Y_{Sa}}{[\sigma_F]}}
$$

试选$K_{Ft}=1.3$

弯曲疲劳强度 $Y_{\varepsilon} = 0.25 + \frac{0.75}{\varepsilon_{\alpha}}$

In [132]:
K_Ft = 1.3

Y_epsilon = 0.25 + 0.75 / epsilon_alpha

print(Y_epsilon)

0.6934123899726368


计算 $\frac{Y_{Fa} Y_{Sa}}{[\sigma_F]}$

由P211表10-5查得

齿型系数 $Y_{Fa1}=2.97, Y_{Fa2}=2.18$

应力修正系数 $Y_{Sa1}=1.52, Y_{Sa2}=1.79$

由P219图10-20c查得小齿轮和大齿轮的齿根弯曲疲劳极限分别为$\sigma_{Flim1}=500\text{MPa}, \sigma_{Flim2}=320\text{MPa}$

由P218图10-18查得弯曲疲劳寿命系数$K_{FN1}=0.9, K_{FN2}=0.95$.

取弯曲疲劳安全系数$S=1.4$,由P217公式10-14得

$[\sigma_F]_1 = \frac{K_{FN1}\sigma_{Flim1}}{S}$

$\frac{Y_{Fa1}Y_{Sa1}}{[\sigma_F]_1}$ 取较大者

In [133]:
# 查表
Y_Fa1 = 2.97
Y_Fa2 = 2.18
Y_Sa1 = 1.52
Y_Sa2 = 1.79
sigma_Flim1 = 500
sigma_Flim2 = 320
K_FN1 = 0.9
K_FN2 = 0.95
S_ = 1.4

sigma_F_1 = K_FN1 * sigma_Flim1 / S_
temp_1 = Y_Fa1 * Y_Sa1 / sigma_F_1

sigma_F_2 = K_FN2 * sigma_Flim2 / S_
temp_2 = Y_Fa2 * Y_Sa2 / sigma_F_2

temp = max(temp_1, temp_2)

print(temp)

0.01797065789473684


In [134]:
m_t = (2*K_Ft*1000*T_1*Y_epsilon/(phi_d*z1_small**2) * temp)**(1/3)
print('mt={:.3f}'.format(m_t))

mt=1.395


### 2.2.6 调整齿轮模数

圆周速度 

$d_1 = m_t z_1 \text{ mm}$

$v = \frac{\pi d_1 n_1}{60\times 1000}$

齿宽 $b = \phi_d d_1$

宽高比 $b/h$

$h=(2h_a^* + c^*)m_t$

$\frac{b}{h} = ?$

In [135]:
c_star = 0.25 # GB

d_1 = m_t * z1_small
print('d_1={:.3f} mm'.format(d_1))
v = pi * d_1 * n1 / (60 * 1000)
b = phi_d * d_1
h = (2*ha_star + c_star)*m_t
b_div_h = b / h
print('v={:.3f} m/s, b/h={:.3f}'.format(v,b_div_h))

d_1=23.709 mm
v=1.806 m/s, b/h=6.044


计算实际载荷系数 $K_F$

根据 $v=1.615 \text{ m/s}$, 7级精度, 由图P206图10-8查得动载系数$K_v = 1.05$

由 $F_{t1} = 2T_1/(0.001d_1)$, 然后$K_AF_{t1}/b=?100 \text{N/mm}$,由 P207 表10-3查得齿间载荷分配系数 $K_{F\alpha}$

由P208表10-4用插值法查得$K_{Hb}=$,结合$b/h$,查P208图10-13,得$K_{F\beta}=?$

则载荷系数为

$$K_F = K_A K_v K_{F\alpha} K_{F\beta}$$

In [136]:
# 查表
K_v = 1.05
F_t1 = 2*T_1 / (0.001 * d_1)
less_than_100 = K_A * F_t1 / b < 100
print('less_than_100:{}'.format(less_than_100))

less_than_100:True


In [137]:
K_Fa = 1
print('b={}'.format(b))

b=18.96754034565983


In [138]:
# 查表
K_Hb = interpolation(40,1.201,80,1.219,b)
K_Hb

1.191535393155547

In [139]:
# 查表
K_Fb = 1.145

K_F = K_A * K_v * K_Fa * K_Fb
print('KF={:.3f}'.format(K_F))

KF=1.202


根据

$$m_F = m_t \sqrt[3]{\frac{K_F}{K_{Ft}}}$$

得到实际载荷系数算得的齿轮模数

以及相应的小齿轮分度圆直径

$$d_F = m_F z_1$$

In [140]:
m_F = m_t * (K_F / K_Ft) **(1/3)
print('m_F = {:.3f}'.format(m_F))

d_F = m_F * z1_small
print('d_F = {:.3f}'.format(d_F))

m_F = 1.359
d_F = 23.100


In [141]:
print('齿面接触疲劳强度计算: m_t = {:.3f}, d_t = {:.3f}'.format(m_t, d_1_H))
print('齿根弯曲疲劳强度计算: m_F = {:.3f}, d_F = {:.3f}'.format(m_F, d_F))

齿面接触疲劳强度计算: m_t = 1.395, d_t = 71.654
齿根弯曲疲劳强度计算: m_F = 1.359, d_F = 23.100


因为一般齿轮模数不应小于2, 所以取$m=2 \text{mm}$.

因为齿轮直径的大小主要取决于齿面接触疲劳强度, 所以取分度圆直径$d_1 = 71.654 \text{mm}$

从而算出小齿轮的齿数 $z_1 = d_1/m$

In [143]:
d1_designed = d_1_H
m_designed = 2

z1_designed = d1_designed / m_designed
print('小齿轮齿数z1_designed={:.3f}'.format(z1_designed))

小齿轮齿数z1_designed=35.827


In [149]:
d1 = d1_designed
m = m_designed
z1 = int(z1_designed)
z2 = int(z1 * i_1)

d1 = z1 * m
d2 = z2 * m
a = (d1 + d2) / 2 # 中心距
b2 = phi_d * d1# 齿轮宽度
b1 = b2 + 5

print('齿数z1={}, z2={}, 模数={}, 压力角={}, 中心距={}, 齿轮宽度b1={} b2={}. 小齿轮选用40Cr(调质), 大齿轮选用45钢(调质). 7级精度'.format(z1, z2, m, phi_d, a, b1, b2))

齿数z1=35, z2=237, 模数=2, 压力角=0.8, 中心距=272.0, 齿轮宽度b1=61.0 b2=56.0. 小齿轮选用40Cr(调质), 大齿轮选用45钢(调质). 7级精度


In [150]:
d2_small = d_2_H

z2_small = int(d2_small / m)
z3 = int(z2_small*i_2)

d2_small = z2_small * m
d3 = z3 * m
a_23 = (d2_small + d3) / 2
b3 = phi_d * d2_small
b2_small = b3 + 5
print('齿数z2small={}, z3={}, 中心距={}, 齿轮宽度b2small={} b3={}'.format(z2_small, z3, a_23, b2_small, b3))

齿数z2small=64, z3=333, 中心距=397.0, 齿轮宽度b2small=107.4 b3=102.4


我们把这些所有齿轮的设计过程封装成一个函数, 按照下述方法调用即可

In [1]:
from design import design_all
# 输入
F_w = 3.2 # kN
v_w = 0.95 # m/s
D = 440 # mm
out1, out2, out_shaft = design_all(F_w, v_w, D)
z1_small,z2_big,m12,b1_small,b2_big,a12 = out1
z2_small,z3_big,m23,b2_small,b3_big,a23 = out2
d1,d2,d3 = out_shaft[0]

工作转速：41.236 rpm
总效率为: 0.816
电机所需功率为：3723.320 W
总传动比为:35.285
i1 = 6.773, i2 = 5.210
轴1的转速为:1455.000, 轴2的转速为:214.831, 轴3的转速为:41.236
P1 = 2949.408 W, P2 = 2832.611 W, P3 = 2720.440 W
epsilon_a = 1.691
试算 d_1t = 36.44 mm
实际 d_1H = 38.58 mm
试算 m_t = 1.40 mm
K_Fb = 1.15
实际 m_F = 1.45 mm
实际 d_1F = 24.71 mm
齿面接触疲劳强度计算: m_H = 2.269, d_H = 38.577
齿根弯曲疲劳强度计算: m_F = 1.453, d_F = 24.706

    主要设计结论:
    齿数z1 = 19
    齿数z2 = 129
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 148 mm
    齿宽b1 = 35 mm
    齿宽b2 = 30 mm
    小齿轮选用40Cr(调质),
    大齿轮选用45钢(调质),
    7级精度
    
epsilon_a = 1.707
试算 d_1t = 66.51 mm
实际 d_1H = 69.33 mm
试算 m_t = 2.33 mm
K_Fb = 1.15
实际 m_F = 2.25 mm
实际 d_1F = 45.01 mm
齿面接触疲劳强度计算: m_H = 3.467, d_H = 69.333
齿根弯曲疲劳强度计算: m_F = 2.250, d_F = 45.007

    主要设计结论:
    齿数z1 = 35
    齿数z2 = 182
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 217 mm
    齿宽b1 = 61 mm
    齿宽b2 = 56 mm
    小齿轮选用40Cr(调质),
    大齿轮选用45钢(调质),
    7级精度
    


### 2.3 联轴器的选择

我们要选用连接电机轴和减速器轴I的联轴器.

电机的轴径为$28$,查机械设计课程设计P191,我们可以选取弹性套筒联轴器<font color='red'>LT4</font>, 公秤转矩为$100$,它大于输入转矩$20$,符合要求.
